# Consultar datos seleccionados de Cassandra e cargalos en Redis (Notebook 3)

## Crea un terceiro notebook.

Este arquivo (```3.CargarRedis.ipynb```) xestiona este paso.

## Desde Cassandra, consulta só dúas columnas — unha será a chave e a outra o valor.

In [ ]:
# Requerimentos
# Python <= 3.11 , por compatibilidade con cassandra.cluster

%pip install redis

In [ ]:
from cassandra.cluster import Cluster

# Supoñemos o Cluster Cassandra na computadora anfitrioa onde está este Notebook (localhost)
#   segundo o arquivo docker-compose.yml empregamos o porto predeterminado (default) 

cluster = Cluster(['127.0.0.1'], port=9042)

try:
    # 2. Conectar e crear a sesión
    session = cluster.connect()
    
    # 3. Proba de conexión: obter o nome do cluster
    row = session.execute("SELECT cluster_name FROM system.local").one()
    print(f"Conectado ó cluster: {row.cluster_name}")

except Exception as e:
    print(f"Non fun quen de conectar a Cassandra. Erro: {e}")


# Deixamos Aberta a conexion ó cluster Cassandra.


In [ ]:
# Gardamos as columnas 'isbn' e 'title' nun dicionario

session.set_keyspace('chaveiro')
resultado = session.execute("SELECT isbn, title FROM libros")

# Pasar o ResultSet a dicionario
datos = {row.isbn: row.title for row in resultado}



In [ ]:
print("Número de elementos da lista 'datos':", len(datos))


## Garda estes pares chave-valor en Redis.

In [ ]:
import redis

# Conectar ao servidor Redis
try:
    r = redis.Redis(host='localhost', port=6379, decode_responses=True)
    print("✅ Conectado a Redis")
except Exception as e:
    print("❌ Erro de conexión a Redis: {e}")



# Gardar cada par no almacén de Redis
for chave, valor in datos.items():
    r.set(chave, valor)

print("Carga en Redis completada.")

## Verifica inserindo e consultando algunhas claves en Redis.

In [ ]:
 # Inserir un rexistro de mostra
r.set('0000000000', '<DUMMY> Libro de proba')

# Consultar o rexistro de mostra, e o primeiro e o derradeiro do Dataset orixinal
print("Proba de consulta en Redis\n==============================")
print(f"Primeiro rexistro:\t {r.get('0195153448')}")
print(f"Derradeiro rexistro:\t {r.get('0767409752')}")
print(f"Novo Rexistro:\t\t {r.get('0000000000')}")